Process and save dataset

In [ ]:
import numpy as np, open3d as o3d
def visualize_sample(strawberry_model_pcd, observation_pcd, Gt):
    filtered_points = np.array(observation_pcd)
    ones = np.ones((filtered_points.shape[0], 1))
    filtered_points_h = np.hstack((filtered_points, ones))
    strawberry_points_transformed = (Gt @ filtered_points_h.T).T[:, :3]

    # Create point clouds from the data
    pcd_strawberry = o3d.geometry.PointCloud()
    pcd_strawberry.points = o3d.utility.Vector3dVector(strawberry_model_pcd)

    pcd_loaded = o3d.geometry.PointCloud()
    pcd_loaded.points = o3d.utility.Vector3dVector(observation_pcd)

    pcd_modified = o3d.geometry.PointCloud()
    pcd_modified.points = o3d.utility.Vector3dVector(strawberry_points_transformed)

    # Create visualizer and add point clouds
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    

    # Add point clouds with different colors
    pcd_strawberry.paint_uniform_color([1, 0, 0])  # Red for strawberry points
    pcd_loaded.paint_uniform_color([0, 0, 1])      # Blue for loaded points
    pcd_modified.paint_uniform_color([0, 1, 0])    # Green for modified points

    vis.add_geometry(pcd_strawberry)
    #vis.add_geometry(pcd_loaded)
    vis.add_geometry(pcd_modified)

    # Run visualization
    vis.run()
    vis.destroy_window()


In [ ]:
import glob, os
import open3d as o3d
import numpy as np
def cal_normal(pcd, radius=0.03, max_nn=30):
    _pcd = o3d.geometry.PointCloud()
    _pcd.points = o3d.utility.Vector3dVector(pcd)
    
    _pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    # o3d.geometry.estimate_normals(_pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    normals = np.asarray(_pcd.normals)
    return normals

def regularize_pcd(pcd, n_points, is_test):
    assert is_test is not None
    if is_test:
        np.random.seed(1)
    idxs = np.random.randint(low=0, high=pcd.shape[0], size=n_points, dtype=np.int64)
    new_pcd = pcd[idxs, :].astype(np.float32)
    return new_pcd
# original loss

orig_strawberry_points = []

strawberry_model_pcd = 'strawberry_fruit_sampled_no_leave.xyz'
# strawberry_model_pcd = 'strawberry_y_positive.xyz'
# gun_model_pcd ='gun.xyz'
with open(strawberry_model_pcd, 'r') as file:
    for line in file:
        x_str, y_str, z_str = line.strip().split()
        orig_strawberry_points.append((float(x_str), float(y_str), float(z_str)))

# Convert to numpy array for processing
orig_strawberry_points = np.array(orig_strawberry_points)
print("orig_strawberry_points", orig_strawberry_points.shape)
# Convert to Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(orig_strawberry_points)




# # # Create visualizer
# vis = o3d.visualization.Visualizer()
# vis.create_window()

# # Add geometry and run
# vis.add_geometry(pcd)
# vis.run()
# vis.destroy_window()

# Get all npz files in the directory
data_dir = '/home/ali/Documents/robotic_picker/data_test/'
dataset_files = glob.glob(os.path.join(data_dir, 'dataset*.npz'))
print(f'Found {len(dataset_files)} dataset files')
                
           





# # # Get all npz files in the directory
# data_dir = '/home/ali/Documents/robotic_picker/data_test/'
# dataset_files = glob.glob(os.path.join(data_dir, 'dataset*.npz'))
# print(f'Found {len(dataset_files)} dataset files')



    
    
    





In [ ]:
# Store results
point_counts = {}
count = 0
# Sort dataset files based on their numeric order
dataset_files = sorted(dataset_files, key=lambda x: int(os.path.basename(x).replace('dataset', '').replace('.npz', '')))

# Load each file and count points

idx = 0
for i in range(len(dataset_files)):
    file_path = dataset_files[i]
    data = np.load(file_path)
    
    Gt = data['y']
    observation_points = data['X']
    if observation_points.shape[0] > 1500:
        print("shape of observation_points", observation_points.shape, " file path ", file_path)
        continue
 
    
    print(file_path, observation_points.shape)

    model_points = np.array(orig_strawberry_points)
    ones = np.ones((model_points.shape[0], 1))
    filtered_points_h = np.hstack((model_points, ones))
    model_transformed = (Gt @ filtered_points_h.T).T[:, :3]
    
    Gt = np.linalg.inv(Gt)
    Gt = Gt[:3]    
    Y_transformed = model_transformed
    Y = model_points
    Y_normal = cal_normal(Y)

    # Process original points
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(observation_points)

    # Get 512 points for original data
    if len(pcd.points) >= 512:
        downsampled_pcd = pcd.farthest_point_down_sample(512)
        X = np.asarray(downsampled_pcd.points)
    else:
        original_points = np.asarray(pcd.points)
        repeats_needed = int(np.ceil(512 / len(original_points)))
        X = np.tile(original_points, (repeats_needed, 1))[:512]

    # Save original sample
    X_normal = cal_normal(X)
    sample_id = os.path.basename(file_path).replace('dataset', '').replace('.npz', '')
    
    # Save original data
    input_data = {
        "src_pcd": X,
        "src_pcd_normal": X_normal,
        "model_pcd": Y,
        "model_pcd_normal": Y_normal,
        "model_pcd_transformed": Y_transformed,
        'transform_gt': Gt,
    }
    output_path = os.path.join(data_dir, f'processed_dataset_{idx}.npz')
    idx += 1
    print(f'Saving original data to {output_path}, {X.shape}')
    np.savez(output_path, **input_data)
    
    # Create and save augmented versions
    num_augmentations = 1
    for aug_idx in range(num_augmentations):
        # Random translation (±1 in any direction)
        translation = np.random.uniform(-1, 1, 3)
        
        # Apply translation only
        X_aug = X + translation
        Y_transformed_aug = Y_transformed + translation
        
        # Calculate normals for augmented point cloud
        X_aug_normal = cal_normal(X_aug)
        
        # Save augmented version
        aug_input_data = {
            "src_pcd": X_aug,
            "src_pcd_normal": X_aug_normal,
            "model_pcd": Y,
            "model_pcd_normal": Y_normal,
            "model_pcd_transformed": Y_transformed_aug,
            'transform_gt': Gt,
        }
        aug_output_path = os.path.join(data_dir, f'processed_dataset_{idx}.npz')
        idx += 1
        print(f'Saving augmented data to {aug_output_path}, {X_aug.shape}')
        np.savez(aug_output_path, **aug_input_data)
  

In [ ]:
import os, glob



data_dir = '/home/ali/Documents/robotic_picker/data_test/'

file_path = os.path.join(data_dir, f'processed_dataset_{50}.npz')

    # Load the data
input_data = np.load(file_path)
# Transform the model points using Gt
model_points = orig_strawberry_points.copy()
ones = np.ones((model_points.shape[0], 1))
model_points_h = np.hstack((model_points, ones))
Gt = input_data['transform_gt']
Gt = np.vstack([Gt, np.array([0, 0, 0, 1])])
model_transformed = (np.linalg.inv(Gt) @ model_points_h.T).T[:, :3]
# Calculate and print the centroid of src_pcd
src_pcd_centroid = np.mean(input_data['src_pcd'], axis=0)
print("Source point cloud centroid:", src_pcd_centroid)

visualize_sample(input_data
                 ['src_pcd'], input_data
                 ['model_pcd_transformed'], np.eye(4))

In [ ]:
import glob
import os
import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans

# Paths
base_path = "log/PointAttN_cd_debug_pcn/all"
import glob, os
import open3d as o3d
import numpy as np
def cal_normal(pcd, radius=0.03, max_nn=30):
    _pcd = o3d.geometry.PointCloud()
    _pcd.points = o3d.utility.Vector3dVector(pcd)
    
    _pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    # o3d.geometry.estimate_normals(_pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    normals = np.asarray(_pcd.normals)
    return normals


def visualize_comparison(src_pcd, src_pcd_inter, batch_idx, sample_idx):
    # Create visualizer
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=f"Batch {batch_idx} Sample {sample_idx}")
    
    # Add source point cloud (red)
    pcd_src = o3d.geometry.PointCloud()
    pcd_src.points = o3d.utility.Vector3dVector(src_pcd)
    pcd_src.paint_uniform_color([1, 0, 0])  # Red
    vis.add_geometry(pcd_src)
    
    # Add interpolated point cloud (blue)
    pcd_inter = o3d.geometry.PointCloud()
    pcd_inter.points = o3d.utility.Vector3dVector(src_pcd_inter)
    pcd_inter.paint_uniform_color([0, 0, 1])  # Blue
    vis.add_geometry(pcd_inter)
    
    # Run visualization
    vis.run()
    vis.destroy_window()


# Process each file in the directory

files = glob.glob(os.path.join(base_path, "batch*_sample*_data.npz"))

for file_path in files:
    # Extract batch and sample numbers from filename
    filename = os.path.basename(file_path)
    batch_idx = int(filename.split('_')[0].replace('batch', ''))
    
    sample_idx = int(filename.split('_')[1].replace('sample', ''))
    
    # Load the data
    data = np.load(file_path)
    src_pcd = data['src_pcd']
    src_pcd_pred = data['xyz']
    unique_points = np.unique(src_pcd_pred, axis=0)
    print(f"Number of unique points: {len(unique_points)}, Total points: {len(src_pcd_pred)}")
    #FPS on src_pcd_inter to get 512 points
    pcd_inter = o3d.geometry.PointCloud()
    pcd_inter.points = o3d.utility.Vector3dVector(src_pcd_pred)
    if len(src_pcd_pred) > 512:
        downsampled_pcd_inter = pcd_inter.farthest_point_down_sample(512)
        src_pcd_pred = np.asarray(downsampled_pcd_inter.points)
    else:
        src_pcd_pred = np.asarray(pcd_inter.points)
        repeats_needed = int(np.ceil(512 / len(src_pcd_pred)))
        src_pcd_pred = np.tile(src_pcd_pred, (repeats_needed, 1))[:512]
    
    print(f"Visualizing batch {batch_idx}, sample {sample_idx}")
    #src_pcd_normal = cal_normal(src_pcd_pred)
    visualize_comparison(src_pcd, src_pcd_pred, batch_idx, sample_idx)
    
    # Optional: wait for user input before showing next visualization
    #input("Press Enter to continue to next sample...")
    
    # # Prepare data for saving in the desired format
    # output_data = {
    #     "src_pcd": src_pcd_pred,  # Use processed 'src_pcd_inter' as 'src_pcd'
    #     "src_pcd_normal": src_pcd_normal,
    #     "model_pcd": data['model_pcd'],
    #     "model_pcd_normal": data['model_pcd_normal'],
    #     "model_pcd_transformed": data['model_pcd_transformed'],
    #     "transform_gt": data['transform_gt']
    # }
    
    # Construct the output file path in data_dir
    
    
    
    # Save the modified data to the new file
    # print(f"Saving modified data to {output_file_path}")
    # np.savez(output_file_path, **output_data)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Number of unique points: 16384, Total points: 16384
Visualizing batch 6, sample 3
Number of unique points: 16384, Total points: 16384
Visualizing batch 1, sample 3
Number of unique points: 16384, Total points: 16384
Visualizing batch 23, sample 3
Number of unique points: 16384, Total points: 16384
Visualizing batch 17, sample 0
Number of unique points: 16384, Total points: 16384
Visualizing batch 17, sample 5
Number of unique points: 16384, Total points: 16384
Visualizing batch 15, sample 6
Number of unique points: 16384, Total points: 16384
Visualizing batch 6, sample 4
Number of unique points: 16384, Total points: 16384
Visualizing batch 12, sample 4
Number of unique points: 16384, Total points: 16384
Visualizing batch 21, sample 1
Number of unique points: 16384, Total points: 16384
Visualizing batch 18, sample 4
Numb

In [ ]:
import os, glob
data_dir = '/home/ali/Documents/robotic_picker/data/'
import numpy as np
import open3d as o3d

dataset_files = glob.glob(os.path.join(data_dir, 'processed_dataset_*.npz'))
print(f'Found {len(dataset_files)} processed dataset files')
# Function to compute chamfer distance
def compute_chamfer_distance(points1, points2):
# Convert to float32 for faster computation
    points1 = points1.astype(np.float32)
    points2 = points2.astype(np.float32)
            
            # Create point clouds
    pcd1 = o3d.geometry.PointCloud()
    pcd2 = o3d.geometry.PointCloud()
    pcd1.points = o3d.utility.Vector3dVector(points1)
    pcd2.points = o3d.utility.Vector3dVector(points2)
            
            # Compute distances
    dist1 = np.asarray(pcd1.compute_point_cloud_distance(pcd2))
    dist2 = np.asarray(pcd2.compute_point_cloud_distance(pcd1))
            
    return np.mean(dist1) + np.mean(dist2)
            
def load_and_visualize_processed_file(file_index):
    """
    Load and visualize a processed dataset file
    Args:
        file_index (int): The index of the file to load
    """
    # Construct file path
    file_path = os.path.join(data_dir, f'processed_dataset_{file_index}.npz')
    
    # Load the data
    data = np.load(file_path)
    model_pcd = data['model_pcd'].copy()
    src_pcd = data['src_pcd'].copy()
    transform_gt = data['transform_gt'].copy()
    
    # Add homogeneous coordinate for visualization
    transform_gt_4x4 = np.vstack([transform_gt, np.array([0, 0, 0, 1])])

    # Try different mirror transformations and compute chamfer distance
    # Define mirror transformations
    mirror_matrices = {
        'no_mirror': np.eye(4),
        'mirror_y': np.array([
        [-1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
        ]),
        'mirror_z': np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
        ]),
        'mirror_yz': np.array([
        [-1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
        ])
    }
    
    best_distance = float('inf')
    best_transform = None
    best_transform_name = 'no_mirror'
    
    # Try each mirror transformation
    for mirror_name, mirror_matrix in mirror_matrices.items():
        # Apply mirroring to the transformation
        test_transform = mirror_matrix @ transform_gt_4x4
        
        # Transform points for comparison
        ones = np.ones((src_pcd.shape[0], 1))
        model_points_h = np.hstack((src_pcd, ones))
        transformed_points = (test_transform @ model_points_h.T).T[:, :3]
        
        # Compute chamfer distance
        distance = compute_chamfer_distance(transformed_points, model_pcd)
        print(f'{mirror_name}: {distance}')
        
        # Update if this is the best transformation so far
        if distance < best_distance:
            best_distance = distance
            best_transform_name = mirror_name
            best_transform = test_transform
    
    # Use the best transformation
    transform_gt_4x4 = best_transform
    print(f'Best mirror transformation: {best_transform_name}')
    
    model_points = np.array(data['model_pcd'])
    ones = np.ones((model_points.shape[0], 1))
    filtered_points_h = np.hstack((model_points, ones))
    model_transformed = (np.linalg.inv(transform_gt_4x4) @ filtered_points_h.T).T[:, :3]

    #visualize_sample(model_transformed, src_pcd, np.eye(4))

    
    # Save the processed data with the new transformation (removing the last row)
    input_data = {
        "src_pcd": data['src_pcd'],
        "src_pcd_normal": data['src_pcd_normal'],
        'model_pcd_transformed': model_transformed,
        "model_pcd": data['model_pcd'],
        "model_pcd_normal": data['model_pcd_normal'],
        'transform_gt': transform_gt_4x4[:3],  # Remove the last row
    }
    
    # Save processed data with sample ID
    output_path = os.path.join(data_dir, f'processed_dataset_{file_index}.npz')
    print(f'Saving processed data to {output_path}')
    np.savez(output_path, **input_data)
        
    # Example usage:
# Process all files in the dataset
for file_index in range(len(dataset_files)):  # Since there are 211 files based on the dataset length
    print(f"\nProcessing file {file_index}")
    load_and_visualize_processed_file(file_index)

In [ ]:
import open3d as o3d
import numpy as np
import glob, os

data_dir = '/home/ali/Documents/robotic_picker/data/predictions'
# Get list of processed dataset files
processed_files = glob.glob(os.path.join(data_dir, '*.npy'))

count = 0


inputs = []
models = []
H_preds = []
H_gts = []

i = 0
for file_path in processed_files:
    print(f"Processing file: {file_path}")
    # Load the NPY file with allow_pickle=True
    data = np.load(file_path, allow_pickle=True).item()  # Convert to dictionary
    loaded_points = data['src_pcd']
    H_pred = data['H_pred']  # Use predicted transformation
    H_gt = data['H_gt']  # Use ground truth transformation
    
    i+=1

    inputs.append(loaded_points)
    models.append(data['model_pcd'])
    H_preds.append(H_pred)
    H_gts.append(H_gt)
    print(H_gt.shape, H_pred.shape)

    # if count == 0:
    #     H_gt = np.vstack([H_gt, np.array([0, 0, 0, 1])])      
        
    #     H_offset = H_pred @ np.linalg.inv(H_gt)
        
    #     H_offset_inv = np.linalg.inv(H_offset)

    
        
    # H_pred = H_offset_inv @ H_pred 

    # Extract translations from transformation matrices
    # Convert to 4x4 matrices by adding homogeneous row
    H_gt_4x4 = np.vstack([H_gt, np.array([0, 0, 0, 1])])
   
    
    # Calculate inverses
    H_gt_inv = np.linalg.inv(H_gt_4x4)
    H_pred_inv = np.linalg.inv(H_pred)
    
    # Extract translations from inverse transformations
    trans_pred = H_pred_inv[:3, 3]
    trans_gt = H_gt_inv[:3, 3]

    # Calculate translation error in cm and round to 2 decimal places
    trans_error_cm = np.round(np.linalg.norm(trans_pred - trans_gt) * 100, 2)
    print(f"Translation error: {trans_error_cm} cm")

    # if count == 100:
    #     visualize_sample(data['model_pcd'], loaded_points, H_pred)
    
    count+=1
   
    # prompt to break
    # input("Press Enter to continue...")
    # if input("Press Enter to continue...") == 'q':
    #     break

            




In [ ]:
import numpy as np, open3d as o3d

model = 'strawberry_fruit_sampled_no_leave.xyz'
# Load points from the file
orig_points = []
with open(model, 'r') as file:
    for line in file:
        x_str, y_str, z_str = line.strip().split()
        orig_points.append((float(x_str), float(y_str), float(z_str)))

# Function to compute chamfer distance
def compute_chamfer_distance(points1, points2):
# Convert to float32 for faster computation
    points1 = points1.astype(np.float32)
    points2 = points2.astype(np.float32)
            
            # Create point clouds
    pcd1 = o3d.geometry.PointCloud()
    pcd2 = o3d.geometry.PointCloud()
    pcd1.points = o3d.utility.Vector3dVector(points1)
    pcd2.points = o3d.utility.Vector3dVector(points2)
            
            # Compute distances
    dist1 = np.asarray(pcd1.compute_point_cloud_distance(pcd2))
    dist2 = np.asarray(pcd2.compute_point_cloud_distance(pcd1))
    return np.mean(dist1) + np.mean(dist2)

                            
def visualize_sample_prediction(strawberry_model_pcd, observation_pcd, H_pred, H_gt):
    filtered_points = np.array(strawberry_model_pcd)
    ones = np.ones((filtered_points.shape[0], 1))
    filtered_points_h = np.hstack((filtered_points, ones))
    ground_truth_transformered = (H_gt @ filtered_points_h.T).T[:, :3]
    prediction_transformered = (H_pred @ filtered_points_h.T).T[:, :3]

    # Calculate and print centroids
    gt_centroid = np.mean(ground_truth_transformered, axis=0)
    pred_centroid = np.mean(prediction_transformered, axis=0)
    centroid_error = np.linalg.norm(gt_centroid - pred_centroid)
    print(f"Ground truth centroid: {gt_centroid}")
    print(f"Prediction centroid: {pred_centroid}")
    print(f"Centroid difference: {centroid_error:.3f}m")

    # Calculate rotation error
    R_gt = H_gt[:3, :3]
    R_pred = H_pred[:3, :3]
    
    # Calculate rotation error using matrix multiplication
    R_error = R_gt @ R_pred.T
    
    # Convert rotation matrix to angle-axis representation
    angle = np.arccos((np.trace(R_error) - 1) / 2)
    rotation_error = np.degrees(angle)
    
    # Handle numerical instabilities
    if np.isnan(rotation_error):
        rotation_error = 0.0
    
    # Calculate translation error
    trans_gt = H_gt[:3, 3]
    trans_pred = H_pred[:3, 3]
    translation_error = np.linalg.norm(trans_gt - trans_pred)
    

    print(f"Rotation error (degrees): {rotation_error:.2f}")
    #Create point clouds and visualization code...
    # observation_mesh_pcd = o3d.geometry.PointCloud()
    # observation_mesh_pcd.points = o3d.utility.Vector3dVector(observation_pcd)

    # prediction_pointcloud = o3d.geometry.PointCloud()
    # prediction_pointcloud.points = o3d.utility.Vector3dVector(prediction_transformered)

    # gt_pointcloud = o3d.geometry.PointCloud()
    # gt_pointcloud.points = o3d.utility.Vector3dVector(ground_truth_transformered)

    # vis = o3d.visualization.Visualizer()
    # vis.create_window()

    # observation_mesh_pcd.paint_uniform_color([1, 0, 0])
    # prediction_pointcloud.paint_uniform_color([0, 0, 1])
    # gt_pointcloud.paint_uniform_color([0, 1, 0])

    # vis.add_geometry(observation_mesh_pcd)
    # vis.add_geometry(prediction_pointcloud)
    

    # vis.run()
    # vis.destroy_window()
    #vis.add_geometry(gt_pointcloud)    

    
    return centroid_error, rotation_error, translation_error

    
centroid_errors = []
rotation_errors = []
translation_errors = []
for i in range(len(inputs)):
    print(f"\nSample {i+1}:")
    
    H_pred = H_preds[i].copy()
    
    H_gt = H_gts[i].copy()
    H_gt = np.vstack([H_gt, np.array([0, 0, 0, 1])])
   
    src_pcd = inputs[i].copy()
    
    
    # # Extract translation from H_pred
    # translation = H_pred[:3, 3].copy()

    # # Define rotation angles to try (in degrees)
    # angles = np.linspace(0, 360, int(37*2))  # every 10 degrees
    
    
    # # Convert inputs[i] to homogeneous coordinates
    # observation_points = np.array(inputs[i].copy())


    # orig_points = np.array(orig_points)
    # ones = np.ones((orig_points.shape[0], 1))
    # orig_points_h = np.hstack((orig_points, ones))

    # best_distance = float('inf')
    # best_mirrored_H = H_pred.copy()
    

    # mirror_matrices = {
    #     'no_mirror': np.eye(4),
    #     'mirror_y': np.array([
    #     [-1, 0, 0, 0],
    #     [0, 1, 0, 0],
    #     [0, 0, 1, 0],
    #     [0, 0, 0, 1]
    #     ]),
    #     'mirror_z': np.array([
    #     [1, 0, 0, 0],
    #     [0, 1, 0, 0],
    #     [0, 0, -1, 0],
    #     [0, 0, 0, 1]
    #     ]),
    #     'mirror_yz': np.array([
    #     [-1, 0, 0, 0],
    #     [0, 1, 0, 0],
    #     [0, 0, -1, 0],
    #     [0, 0, 0, 1]
    #     ])
    # }
    
    # best_distance = float('inf')
    # best_transform = None
    # best_transform_name = 'no_mirror'
    
    # # Try each mirror transformation
    # for mirror_name, mirror_matrix in mirror_matrices.items():
    #     # Apply mirroring to the transformation
    #     test_transform = mirror_matrix @ H_pred
        
    #     # Transform points for comparison
    #     ones = np.ones((src_pcd.shape[0], 1))
    #     model_points_h = np.hstack((src_pcd, ones))
    #     transformed_points = (test_transform @ model_points_h.T).T[:, :3]
        
    #     # Compute chamfer distance
    #     distance = compute_chamfer_distance(transformed_points, orig_points)
    #     print(f'{mirror_name}: {distance}')
        
    #     # Update if this is the best transformation so far
    #     if distance < best_distance:
    #         best_distance = distance
    #         best_transform_name = mirror_name
    #         best_transform = test_transform

    

   

    # print(f"Best chamfer distance: {best_distance}")
    # print(f"Best mirror transformation: {best_transform_name}")

    # # Update H_pred with the best mirrored transformation
    # H_pred = best_mirrored_H
    
    print(H_pred.shape, H_gt.shape)



    H_pred = np.linalg.inv(H_pred)
    H_gt = np.linalg.inv(H_gt)
   
    centroid_error, rotation_error, translation_error = visualize_sample_prediction(
        orig_points, src_pcd, 
        H_pred, H_gt
    )
    
    centroid_errors.append(centroid_error)
    rotation_errors.append(rotation_error)
    translation_errors.append(translation_error)
    
    # Calculate success rates for different thresholds

translation_thresholds = [0.02, 0.02, 0.01, 0.01]  # in meters
rotation_thresholds = [20, 10, 20, 10]  # in degrees

for t_threshold, r_threshold in zip(translation_thresholds, rotation_thresholds):
 # Count samples within both thresholds
    num_within_threshold = sum(1 for t, r in zip(translation_errors, rotation_errors) 
        if t < t_threshold and r < r_threshold)
        
        # Calculate percentage
    percentage_within_threshold = (num_within_threshold / len(translation_errors)) * 100
        
    print(f"Success rate for ({t_threshold*100}cm, {r_threshold}°): {percentage_within_threshold:.2f}%")

print(f"Centroid Error: {centroid_error:.4f}m, Rotation Error: {rotation_error:.2f} degrees, Translation Error: {translation_error*100:.2f} cm")

print("\nSummary Statistics:")
print(models[0].shape)
print(f"Average centroid error: {np.mean(centroid_errors):.3f}m")
print(f"Average rotation error: {np.mean(rotation_errors):.2f} degrees")
print(f"Average translation error: {np.mean(translation_errors) * 10:.3f}cm")

In [ ]:
print(inputs[0].shape)

In [ ]:
r_orig_strawberry_points = []
orig_strawberry_points = []
import numpy as np

with open('/home/ali/Downloads/strawberry_fruit_sampled.xyz', 'r') as file:
    for line in file:
        x_str, y_str, z_str = line.strip().split()
        r_orig_strawberry_points.append((float(x_str), float(y_str), float(z_str)))



# Convert to numpy arrays
points1 = np.array(r_orig_strawberry_points)
points2 = np.array(orig_strawberry_points)

# Create rotation matrix for 90 degrees around x-axis
rotation_x = o3d.geometry.get_rotation_matrix_from_xyz([np.pi/2, 0, 0])

# Rotate points2
points2_rotated = points2 @ rotation_x.T

# Visualize both point clouds
visualize_sample(points1, points2_rotated, np.eye(4))

In [ ]:
import numpy as np
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from ultralytics import YOLO
import os
import glob
import cv2
import open3d as o3d


def segment_points(masks_list, points_reshaped):
    filtered_points = {}
    for i, mask in enumerate(masks_list):
        mask_bool = mask > 0
        filtered = points_reshaped[mask_bool]
        if len(filtered) > 0:
            # Compute centroid and distances to it
            print(f"number of segmented points {len(filtered)}")
            centroid = np.mean(filtered, axis=0)
            distances = np.linalg.norm(filtered - centroid, axis=1)
            # Use median absolute deviation to set a robust threshold
            median_dist = np.median(distances)
            mad = np.median(np.abs(distances - median_dist))
            # Set threshold as median + 3 * MAD (avoid zero MAD)
            threshold = median_dist + 3 * (mad if mad > 0 else 1e-6)
            # Keep only inliers that are not too far from the centroid
            inliers = filtered[distances <= threshold]
            print(f"number of inliers {len(inliers)}")
            if len(inliers) > 0:
                filtered_points[i] = inliers

    return filtered_points



def segment_pointcloud(frame, points):
    # yolo setup
    model_path = 'yolo_model.pt'
    yolo_model = YOLO(model_path)

    # sam2 setup        
    checkpoint =  "sam2_hiera_small.pt"
    model_cfg = "sam2_hiera_s.yaml"
    predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))
    # YOLO detection and SAM2 segmentation (using existing code)
    results = yolo_model.track(source=frame, persist=True, conf=0.6, verbose=False)
    if results[0].boxes.id is None:
        print("No object detected")
    else:
        masks_list = []
        bboxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        ids = results[0].boxes.id.cpu().numpy().astype(int)
        confidences = results[0].boxes.conf.cpu().numpy().astype(float)

        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            predictor.set_image(frame)
            for box in bboxes:
                input_box = np.array(box).reshape(1, 4)            
                masks, _, _ = predictor.predict(box=input_box, multimask_output=False)
                mask = (masks > 0).astype(np.uint8) * 255
                masks_list.append(mask[0])

        # Visualize segmented image
        visualization_frame = frame.copy()
        for mask in masks_list:
            # Apply mask overlay on the image
            colored_mask = np.zeros_like(frame)
            colored_mask[mask > 0] = [0, 255, 0]  # Green overlay
            visualization_frame = cv2.addWeighted(visualization_frame, 1, colored_mask, 0.5, 0)
        

        # Filter and visualize point cloud
        print(f"masks_list len: {len(masks_list)}")
        points_reshaped = points.reshape(480, 640, 3)
        #filtered_points = np.zeros_like(points_reshaped)
        filtered_points = segment_points(masks_list, points_reshaped)
        # for mask in masks_list:
        #     print("loop start")
        #     # Reshape points to match image dimensions (480, 640, 3)
            
            
        #     # Get points corresponding to the mask
            
        #     if 'filtered_points' not in locals():
        #         filtered_points = points_reshaped[mask > 0]
        #     else:
        #         filtered_points = np.vstack((filtered_points, points_reshaped[mask > 0]))
        return filtered_points
            
# Get all files from both directories
rgb_dir = "/home/netbot/Documents/Malak/Intel Realsense/strawberry_extracted_rgb"
pointcloud_dir = "/home/netbot/Documents/Malak/Intel Realsense/strawberry_extracted_pointcloud"
segmented_pcl_dir="/home/netbot/Documents/Malak/sam2/sam2/segmented_pointcloud"

rgb_files = sorted(glob.glob(os.path.join(rgb_dir, "*.png")))
pointcloud_files = sorted(glob.glob(os.path.join(pointcloud_dir, "*.txt")))

# Process each pair of files
for rgb_file, pc_file in zip(rgb_files, pointcloud_files):
    # Extract IDs to ensure they match
    rgb_id = os.path.basename(rgb_file).split('_')[-1].split('.')[0]
    pc_id = os.path.basename(pc_file).split('_')[-1].split('.')[0]
    
    if rgb_id == pc_id:
        # Load the files
        frame = cv2.imread(rgb_file)
        points = np.loadtxt(pc_file)
        
        # Process the files
        filtered_points = segment_pointcloud(frame, points)
        
        # For each segmented point cloud in the dictionary
        for pcd_id, points in filtered_points.items():
            # Create point cloud object
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(points)
            
            # Create unique filename using scene_id and point cloud id
            output_file = os.path.join(segmented_pcl_dir, f"segmented_{pc_id}_{pcd_id}.ply")
            
            # Save the segmented point cloud
            o3d.io.write_point_cloud(output_file, pcd)
            
            # Optionally visualize each point cloud
            # o3d.visualization.draw_geometries([pcd])

In [ ]:
import numpy as np
import open3d as o3d

orig_strawberry_points = []
strawberry_model_pcd = '/home/ali/Downloads/strawberry_fruit_sampled.xyz'
with open(strawberry_model_pcd, 'r') as file:
        for line in file:
                x_str, y_str, z_str = line.strip().split()
                orig_strawberry_points.append((float(x_str), float(y_str), float(z_str)))

points_array = np.array(orig_strawberry_points)
# Remove the top part of y-axis (e.g. leaves) by keeping only points with y lower than a threshold
threshold = 0.016  # adjust the threshold value as needed
filtered_points = points_array[points_array[:, 1] < threshold]

# Additional filtering based on a plane (only for points with negative x and positive y)
# The plane is assumed to be tilted around z with the following parameters:
#   Plane equation: y = tan(angle) * x + offset
#   Points that lie above this plane (i.e., y value greater than the plane value) will be removed.
plane_angle_deg = 90 # angle in degrees (editable)
plane_angle = np.deg2rad(plane_angle_deg)
plane_offset = 0.020   # offset along y-axis (editable)
slope = np.tan(plane_angle)

# Create a mask to keep points
# Only apply this filter for points in the quadrant: x < 0 and y > 0.
mask = np.ones(filtered_points.shape[0], dtype=bool)
for i, (x, y, z) in enumerate(filtered_points):
        if x < 0 and y > 0:
                if y > (slope * x + plane_offset):
                        mask[i] = False

final_points = filtered_points[mask]



print("Plane angle (degrees):", plane_angle_deg)
print("Plane offset:", plane_offset)
# Clone points with positive y by rotating them around the z-axis
positive_y_mask = final_points[:, 1] > 0
points_positive_y = final_points[positive_y_mask]

#Define a rotation matrix around z (e.g. 180° rotation to mirror these points)
theta = np.pi  # 180 degrees in radians
Ry = np.array([[np.cos(theta), 0, np.sin(theta)],
                           [0, 1, 0],
                           [-np.sin(theta), 0, np.cos(theta)]])
cloned_points = (Ry @ points_positive_y.T).T

# Append the cloned points to final_points
final_points = np.vstack([final_points, cloned_points])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(final_points)

axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
o3d.visualization.draw_geometries([pcd, axis])

np.savetxt('strawberry_fruit_sampled_no_leave.xyz', final_points, fmt='%.8f')
print("Final points saved as strawberry_fruit_sampled_no_leave.xyz")
print("Cloned positive y points around y-axis. Total points:", final_points.shape[0])

In [ ]:
import numpy as np
import open3d as o3d




# Path to the xyz file
file_path = 'strawberry_fruit_sampled_no_leave.xyz'

# Load the point data (assuming the file contains three columns: x, y, z)
points = np.loadtxt(file_path)

# Create an Open3D PointCloud object and assign the points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Optionally, you can estimate normals or set colors, for example:
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
# pcd.paint_uniform_color([0.1, 0.8, 0.1])  # Greenish color

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])




In [ ]:
import os
import glob
import tkinter as tk
import numpy as np
import open3d as o3d
from scipy.optimize import minimize
from scipy.spatial import cKDTree
 
# (Keep existing code to load strawberry model, orig_points, etc.)
 
orig_strawberry_points = []
strawberry_model_pcd = 'strawberry_fruit_sampled_no_leave.xyz'
with open(strawberry_model_pcd, 'r') as file:
    for line in file:
        x_str, y_str, z_str = line.strip().split()
        orig_strawberry_points.append((float(x_str), float(y_str), float(z_str)))

orig_strawberry_points = np.array(orig_strawberry_points)
print("orig_strawberry_points", orig_strawberry_points.shape)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(orig_strawberry_points)

# Set model color to red and copy points
pcd.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(orig_strawberry_points), 1)))
orig_points = np.asarray(pcd.points).copy()

# Load an initial partial point cloud
partial_pcd = o3d.io.read_point_cloud("segmented_real_pointcloud/segmented_0002_0.ply")
partial_points = np.asarray(partial_pcd.points)
partial_pcd.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (partial_points.shape[0], 1)))

# --- Transformation: Align pcd points to the centroid of partial_pcd and rotate around x by 90 degrees ---

# Compute centroids of the strawberry model and the current partial point cloud.
strawberry_centroid = np.mean(orig_strawberry_points, axis=0)
partial_centroid = np.mean(partial_points, axis=0)

# Calculate translation needed to align the strawberry centroid to the partial centroid.
tx, ty, tz = partial_centroid - strawberry_centroid

# Define rotation angles in degrees; here, 90 degrees around the x-axis.
rx, ry, rz = 90, 0, 0

# Generate the rotation matrix for the given angles.
R = o3d.geometry.get_rotation_matrix_from_xyz((np.deg2rad(rx), np.deg2rad(ry), np.deg2rad(rz)))

# Apply the rotation followed by the translation to the strawberry points.
transformed_points = (R @ orig_strawberry_points.T).T + np.array([tx, ty, tz])
pcd.points = o3d.utility.Vector3dVector(transformed_points)
pcd.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))

print("Applied translation (tx, ty, tz):", (tx, ty, tz))
print("Applied rotation (rx, ry, rz in degrees):", (rx, ry, rz))


coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
 
pcd_display = o3d.geometry.PointCloud()
pcd_display.points = o3d.utility.Vector3dVector(transformed_points)
pcd_display.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
 
# Create a global visualizer (initial title will be used for the first partial)
vis = o3d.visualization.Visualizer()
# vis.create_window("Visualizer - segmented_0000_0", width=960, height=1080, left=2000, top=10)
vis.create_window("Visualizer - segmented_0000_0", width=1920, height=2160, left=2000)
vis.add_geometry(pcd_display)
vis.add_geometry(partial_pcd)
#vis.add_geometry(coordinate_frame)
 
transformation = np.eye(4)
 
def apply_transformation():
    global transformation, pcd_display
    tx = slider_tx.get()
    ty = slider_ty.get()
    tz = slider_tz.get()
    rx = np.deg2rad(slider_rx.get())
    ry = np.deg2rad(slider_ry.get())
    rz = np.deg2rad(slider_rz.get())
    R = o3d.geometry.get_rotation_matrix_from_xyz((rx, ry, rz))
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = [tx, ty, tz]
    transformation = T
    transformed_points = (R @ orig_points.T).T + np.array([tx, ty, tz])
    pcd_display.points = o3d.utility.Vector3dVector(transformed_points)
    pcd_display.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
    vis.update_geometry(pcd_display)
    print("Applied transformation:\n", T)
 

def move_to_centroid():
    # Compute initial guess.
    strawberry_centroid = np.mean(orig_points, axis=0)
    partial_centroid = np.mean(partial_points, axis=0)
    translation_init = partial_centroid - strawberry_centroid
    init_tx, init_ty, init_tz = translation_init
    init_rx, init_ry, init_rz = np.deg2rad(90), 0, 0

    

    # Define the objective function (Chamfer distance with a z-penalty).
    def chamfer_distance(params):
        tx, ty, tz, rx, ry, rz = params
        R = o3d.geometry.get_rotation_matrix_from_xyz((rx, ry, rz))
        transformed = (R @ orig_points.T).T + np.array([tx, ty, tz])
        tree_partial = cKDTree(partial_points)
        dists1, _ = tree_partial.query(transformed)
        tree_trans = cKDTree(transformed)
        dists2, _ = tree_trans.query(partial_points)
        chamfer = np.mean(dists1**2) + np.mean(dists2**2)
        penalty = 1000 * ((tz - init_tz)**2)  # Strongly penalize z shift
        return chamfer + penalty
    
    # Run optimization starting from the initial guess.
    x0 = [init_tx, init_ty, init_tz, init_rx, init_ry, init_rz]
    res = minimize(chamfer_distance, x0, method='Nelder-Mead', options={'maxiter': 100})
    optimal = res.x
    print("Optimization result:", optimal)

    slider_tx.set(optimal[0])
    slider_ty.set(optimal[1])
    slider_tz.set(optimal[2])
    slider_rx.set(np.rad2deg(optimal[3]))
    slider_ry.set(np.rad2deg(optimal[4]))
    slider_rz.set(np.rad2deg(optimal[5]))
 
    apply_transformation()
    print("Moved strawberry model to partial pointcloud's centroid.")
 
def save_transformation():
    current_filename=os.path.basename(partial_files[partial_index-1])
    identifier = current_filename.replace("segmented_", "").replace(".ply", "")
    
    os.makedirs("transformation_matrix", exist_ok=True)
    np.save(f"transformation_matrix/transformation_matrix_{identifier}.npy", transformation)
    transformed_points = (transformation[:3, :3] @ orig_points.T).T + transformation[:3, 3]
    pcd_save = o3d.geometry.PointCloud()
    pcd_save.points = o3d.utility.Vector3dVector(transformed_points)
    pcd_save.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
    os.makedirs("transformation_strawberry_model", exist_ok=True)
    o3d.io.write_point_cloud(f"transformation_strawberry_model/transformation_strawberry_model_{identifier}.ply", pcd_save)
    print("Saved transformation matrix and model for", identifier)
 
# Process All function remains unchanged
def process_all():
    folder = "segmented_real_pointcloud"
    pattern = os.path.join(folder, "segmented_*.ply")
    files = sorted(glob.glob(pattern))
    for filepath in files:
        print("Processing", filepath)
        current_partial_pcd = o3d.io.read_point_cloud(filepath)
        current_points = np.asarray(current_partial_pcd.points)
        current_partial_pcd.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (current_points.shape[0], 1)))
        global partial_points, partial_pcd
        partial_points = current_points
        try:
            vis.remove_geometry(partial_pcd)
        except:
            pass
        partial_pcd = current_partial_pcd
        vis.add_geometry(partial_pcd)
        strawberry_centroid = np.mean(orig_points, axis=0)
        partial_centroid = np.mean(partial_points, axis=0)
        translation = partial_centroid - strawberry_centroid
        slider_tx.set(translation[0])
        slider_ty.set(translation[1])
        slider_tz.set(translation[2])
        slider_rx.set(90)
        slider_ry.set(0)
        slider_rz.set(0)
        apply_transformation()
       
        identifier = os.path.splitext(os.path.basename(filepath))[0]
        # identifier = os.path.basename(filename)
 
        save_transformation(identifier)
        vis.poll_events()
 
# Create a sorted list of partial pointcloud files and an index for going to the next partial.
partial_files = sorted(glob.glob(os.path.join("segmented_real_pointcloud", "segmented_*.ply")))
partial_index = 1   # Since the first file is already loaded
 
def next_partial():
    global partial_files, partial_index, partial_pcd, partial_points, vis
    if partial_index >= len(partial_files):
        print("No more partial pointclouds.")
        return
    filename = partial_files[partial_index]
    partial_index += 1
    print("Loading", filename)
    # Read the new partial
    new_partial = o3d.io.read_point_cloud(filename)
    new_partial_points = np.asarray(new_partial.points)
    new_partial.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (new_partial_points.shape[0], 1)))
    partial_points = new_partial_points
    try:
        vis.remove_geometry(partial_pcd)
    except:
        pass
    partial_pcd = new_partial
    # Update sliders so that the strawberry model is centered on the partial
    strawberry_centroid = np.mean(orig_points, axis=0)
    partial_centroid = np.mean(partial_points, axis=0)
    translation = partial_centroid - strawberry_centroid
    slider_tx.set(translation[0])
    slider_ty.set(translation[1])
    slider_tz.set(translation[2])
    slider_rx.set(90)
    slider_ry.set(0)
    slider_rz.set(0)
    apply_transformation()
    # Re-create the visualizer window with the new title (the title now shows the partial file name)
    new_title = f"Visualizer - {os.path.basename(filename)}"
 
    vis.destroy_window()
    vis.create_window(new_title, width=1920, height=2160, left=2000)
    vis.add_geometry(pcd_display)
    vis.add_geometry(partial_pcd)
    vis.add_geometry(coordinate_frame)
    print("Updated visualizer with new title.")
 
def update_visualizer():
    vis.poll_events()
    vis.update_renderer()
    root.after(50, update_visualizer)
 
def create_slider_frame(parent, label_text, slider_from, slider_to, resolution):
    frame = tk.Frame(parent)
    frame.pack(pady=2)
    tk.Label(frame, text=label_text).pack(side=tk.LEFT)
    slider = tk.Scale(frame, from_=slider_from, to=slider_to, resolution=resolution,
                      orient=tk.HORIZONTAL, length=200, command=lambda x: apply_transformation())
    slider.pack(side=tk.LEFT)
    btn_minus = tk.Button(frame, text="-", command=lambda: slider.set(slider.get() - resolution))
    btn_minus.pack(side=tk.LEFT, padx=2)
    btn_plus = tk.Button(frame, text="+", command=lambda: slider.set(slider.get() + resolution))
    btn_plus.pack(side=tk.LEFT, padx=2)
    return slider
 
root = tk.Tk()
root.title("Strawberry Transformation")
 
slider_tx = create_slider_frame(root, "Translation X", -1, 1, 0.002)
slider_ty = create_slider_frame(root, "Translation Y", -1, 1, 0.002)
slider_tz = create_slider_frame(root, "Translation Z", -1, 1, 0.002)
slider_rx = create_slider_frame(root, "Rotation X (deg)", -180, 180, 0.5)
slider_ry = create_slider_frame(root, "Rotation Y (deg)", -180, 180, 0.5)
slider_rz = create_slider_frame(root, "Rotation Z (deg)", -180, 180, 0.5)
 
btn_move = tk.Button(root, text="Move to Centroid", command=move_to_centroid)
btn_move.pack(pady=5)
 
 
 
 
btn_save = tk.Button(root, text="Save Transformation", command=lambda: save_transformation())
btn_save.pack(pady=5)
btn_process_all = tk.Button(root, text="Process All Partials", command=process_all)
btn_process_all.pack(pady=5)
 
# New button to go to the next partial pointcloud
btn_next = tk.Button(root, text="Next Partial", command=next_partial)
btn_next.pack(pady=5)
 
root.after(50, update_visualizer)
root.mainloop()

In [ ]:
import glob
import os
import open3d as o3d
import numpy as np

# Directory containing the .ply files for output and input
ply_directory = "transformation_strawberry_model"
in_directory = "segmented_real_pointcloud"

# Get list of output .ply files in the ply_directory
ply_files = glob.glob(os.path.join(ply_directory, "*.ply"))

# Loop through each output file, find the corresponding input file, and visualize both
all_points_list = []  # To collect all points for global statistics

for out_file in ply_files:
    basename = os.path.basename(out_file)
    # Expected format: transformation_strawberry_model_0002_0.ply
    parts = basename.split('_')
    # The identifier is the 4th and 5th parts (e.g., "0002" and "0.ply")
    if len(parts) < 5:
        print(f"Skipping file with unexpected format: {basename}")
        continue
    identifier = parts[3] + '_' + parts[4].replace('.ply', '')
    in_filename = f"segmented_{identifier}.ply"
    in_file = os.path.join(in_directory, in_filename)
    
    if not os.path.exists(in_file):
        print(f"Input file not found for {basename}: expected {in_filename}")
        continue

    print(f"\nVisualizing:\n Output: {out_file}\n Input: {in_file}")
    in_pcd = o3d.io.read_point_cloud(in_file)
    out_pcd = o3d.io.read_point_cloud(out_file)
    points = np.asarray(in_pcd.points)
    all_points_list.append(points)
    
    # Compute and print per-file statistics for all axes
    mean_vals = np.mean(points, axis=0)
    std_vals = np.std(points, axis=0)
    print(f"Stats for {in_filename}:")
    print(f"  Mean (x,y,z): {mean_vals}")
    print(f"  Standard deviation (x,y,z): {std_vals}")
    
    # # Robust z filtering using median and MAD
    # z_vals = points[:, 2]
    # z_median = np.median(z_vals)
    # mad = np.median(np.abs(z_vals - z_median))
    # mad_threshold = 1.5 * mad if mad > 0 else 0.01

    # mask = np.abs(z_vals - z_median) < mad_threshold
    in_filtered_points = points.copy()
    print(f"  Initial number of points: {in_filtered_points.shape[0]}")

    # Additional filtering step: enforce that the standard deviation in z only is below the target value.
    target_std = 0.003
    current_std = np.std(in_filtered_points[:, 2])
    iter_count = 0
    max_iter = 1000
    outlier_points = []  # To collect points that got filtered out
    print(f"  Initial std(z): {current_std:.8f}, target std(z): {target_std:.8f}")
    while current_std > target_std and in_filtered_points.shape[0] > 1 and iter_count < max_iter:
        # Remove the point with the maximum deviation from the median along z
        z_vals_filtered = in_filtered_points[:, 2]
        deviation = np.abs(z_vals_filtered - np.median(z_vals_filtered))
        remove_index = np.argmax(deviation)
        # Save the removed point as an outlier
        outlier_points.append(in_filtered_points[remove_index])
        in_filtered_points = np.delete(in_filtered_points, remove_index, axis=0)
        current_std = np.std(in_filtered_points[:, 2])
        iter_count += 1

    print(f"After additional filtering: kept {in_filtered_points.shape[0]} inliers with std(z) = {current_std:.8f}")

    # Create point clouds for visualization with different colors
    in_filtered_pcd = o3d.geometry.PointCloud()
    in_filtered_pcd.points = o3d.utility.Vector3dVector(in_filtered_points)
    in_filtered_pcd.paint_uniform_color([0, 1, 0])  # Green for inliers

    outlier_points = np.array(outlier_points)
    in_outlier_pcd = o3d.geometry.PointCloud()
    if outlier_points.size > 0:
        in_outlier_pcd.points = o3d.utility.Vector3dVector(outlier_points)
    in_outlier_pcd.paint_uniform_color([1, 0, 0])  # Red for outliers
    out_pcd.paint_uniform_color([0, 0, 1])  # Blue for output

    # Visualize the filtered inliers and the removed outliers together.
    o3d.visualization.draw_geometries([in_filtered_pcd, out_pcd])


In [ ]:
import cv2
import numpy as np
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from ultralytics import YOLO
import os
import glob
import open3d as o3d
model_path = 'yolo_model.pt'
yolo_model = YOLO(model_path)
# sam2 setup        
checkpoint =  "sam2_hiera_small.pt"
model_cfg = "sam2_hiera_s.yaml"
predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))

def segment_pointcloud(frame, points):
    # YOLO detection and SAM2 segmentation (using existing code)
    results = yolo_model.predict(source=frame, conf=0.6, verbose=True)
    if len(results[0].boxes) == 0:
        print("No object detected")
        return None
    else:
        masks_list = []
        bboxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        confidences = results[0].boxes.conf.cpu().numpy().astype(float)

        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            predictor.set_image(frame)
            for box in bboxes:
                input_box = np.array(box).reshape(1, 4)            
                masks, _, _ = predictor.predict(box=input_box, multimask_output=False)
                mask = (masks > 0).astype(np.uint8) * 255
                masks_list.append(mask[0])

        # Visualize segmented image
        visualization_frame = frame.copy()
        for mask in masks_list:
            # Apply mask overlay on the image
            colored_mask = np.zeros_like(frame)
            colored_mask[mask > 0] = [0, 255, 0]  # Green overlay
            visualization_frame = cv2.addWeighted(visualization_frame, 1, colored_mask, 0.5, 0)
        
        # Save the segmented visualization image
        cv2.imwrite("segmented_visualization.png", visualization_frame)
        print("Segmented image saved as segmented_visualization.png")

        # Filter and visualize point cloud
        print(f"masks_list len: {len(masks_list)}")
        points_reshaped = points.reshape(480, 848, 3)
        for mask in masks_list:
            print("loop start")
            if 'filtered_points' not in locals():
                filtered_points = points_reshaped[mask > 0]
            else:
                filtered_points = np.vstack((filtered_points, points_reshaped[mask > 0]))
        return filtered_points

def extra_filter(points):
    if points.shape[0] > 0:
        # Compute centroid and distances to it
        
        centroid = np.mean(points, axis=0)
        distances = np.linalg.norm(points - centroid, axis=1)
        # Use median absolute deviation to set a robust threshold
        median_dist = np.median(distances)
        mad = np.median(np.abs(distances - median_dist))
            # Set threshold as median + 3 * MAD (avoid zero MAD)
        threshold = median_dist + 3 * (mad if mad > 0 else 1e-6)
            # Keep only inliers that are not too far from the centroid
        inliers = points[distances <= threshold]
        print(f"number of inliers {len(inliers)}")
        if len(inliers) > 0:
            filtered_points = inliers.copy()
            return filtered_points
    return None

Labeling

In [ ]:
import os
import glob
import tkinter as tk
import numpy as np
import open3d as o3d
from scipy.optimize import minimize
from scipy.spatial import cKDTree
 
# (Keep existing code to load strawberry model, orig_points, etc.)
 
orig_strawberry_points = []
# strawberry_model_pcd = 'strawberry_fruit_sampled.xyz'
strawberry_model_pcd = 'strawberry_fruit_sampled_no_leave.xyz'
with open(strawberry_model_pcd, 'r') as file:
    for line in file:
        x_str, y_str, z_str = line.strip().split()
        orig_strawberry_points.append((float(x_str), float(y_str), float(z_str)))
 
orig_strawberry_points =  np.array(orig_strawberry_points)
#orig_strawberry_points = orig_strawberry_points[orig_strawberry_points[:, 2] > 0]
print("orig_strawberry_points", orig_strawberry_points.shape)
 
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(orig_strawberry_points)
 
# Set model color to red and copy points
pcd.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(orig_strawberry_points), 1)))
orig_points = np.asarray(pcd.points).copy()
 
# Load an initial partial point cloud
# partial_pcd = o3d.io.read_point_cloud("segmented_pointcloud_separated/segmented_0000_0.ply")
# partial_pcd = o3d.io.read_point_cloud("front_orientation3_separated/segmented_0000_0.ply")
partial_pcd = o3d.io.read_point_cloud("data/segmented/filtered_points_0202_detection_0.ply")
# partial_points = np.load("filtered_points2.npy")
# partial_pcd = o3d.geometry.PointCloud()
# partial_pcd.points = o3d.utility.Vector3dVector(partial_points)


partial_points = np.asarray(partial_pcd.points)
partial_pcd.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (partial_points.shape[0], 1)))
 
# --- Transformation: Align pcd points to the centroid of partial_pcd and rotate around x by 90 degrees ---
 
# Compute centroids of the strawberry model and the current partial point cloud.
strawberry_centroid = np.mean(orig_strawberry_points, axis=0)
partial_centroid = np.mean(partial_points, axis=0)
 
# Calculate translation needed to align the strawberry centroid to the partial centroid.
tx, ty, tz = partial_centroid - strawberry_centroid
 
# Define rotation angles in degrees; here, 90 degrees around the x-axis.
rx, ry, rz = 90, 0, 0
 
# Generate the rotation matrix for the given angles.
R = o3d.geometry.get_rotation_matrix_from_xyz((np.deg2rad(rx), np.deg2rad(ry), np.deg2rad(rz)))
 
# Apply the rotation followed by the translation to the strawberry points.
transformed_points = (R @ orig_strawberry_points.T).T + np.array([tx, ty, tz])
pcd.points = o3d.utility.Vector3dVector(transformed_points)
pcd.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
 
print("Applied translation (tx, ty, tz):", (tx, ty, tz))
print("Applied rotation (rx, ry, rz in degrees):", (rx, ry, rz))
 
 
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
 
pcd_display = o3d.geometry.PointCloud()
pcd_display.points = o3d.utility.Vector3dVector(transformed_points)
pcd_display.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
 
# Create a global visualizer (initial title will be used for the first partial)
vis = o3d.visualization.Visualizer()
# vis.create_window("Visualizer - segmented_0000_0", width=960, height=1080, left=2000, top=10)
vis.create_window("Visualizer - segmented_0100_0", width=1920, height=2160, left=2000)
vis.add_geometry(pcd_display)
vis.add_geometry(partial_pcd)
#vis.add_geometry(coordinate_frame)
 
transformation = np.eye(4)
 

# folder = "segmented_pointcloud_separated"
# folder = "front_orientation3_separated"
folder = "data/segmented"

def apply_transformation():
    global transformation, pcd_display
    tx = slider_tx.get()
    ty = slider_ty.get()
    tz = slider_tz.get()
    rx = np.deg2rad(slider_rx.get())
    ry = np.deg2rad(slider_ry.get())
    rz = np.deg2rad(slider_rz.get())
    R = o3d.geometry.get_rotation_matrix_from_xyz((rx, ry, rz))
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = [tx, ty, tz]
    transformation = T
    transformed_points = (R @ orig_points.T).T + np.array([tx, ty, tz])
    pcd_display.points = o3d.utility.Vector3dVector(transformed_points)
    pcd_display.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
    vis.update_geometry(pcd_display)
    print("Applied transformation:\n", T)
 
 
def move_to_centroid():
    # Compute initial guess.
    strawberry_centroid = np.mean(orig_points, axis=0)
    partial_centroid = np.mean(partial_points, axis=0)
    translation_init = partial_centroid - strawberry_centroid
    init_tx, init_ty, init_tz = translation_init
    init_rx, init_ry, init_rz = np.deg2rad(90), 0, 0
 
    
 
    # Define the objective function (Chamfer distance with a z-penalty).
    def chamfer_distance(params):
        tx, ty, tz, rx, ry, rz = params
        R = o3d.geometry.get_rotation_matrix_from_xyz((rx, ry, rz))
        transformed = (R @ orig_points.T).T + np.array([tx, ty, tz])
        tree_partial = cKDTree(partial_points)
        dists1, _ = tree_partial.query(transformed)
        tree_trans = cKDTree(transformed)
        dists2, _ = tree_trans.query(partial_points)
        chamfer = np.mean(dists1**2) + np.mean(dists2**2)
        penalty = 1000 * ((tz - init_tz)**2)  # Strongly penalize z shift
        return chamfer + penalty
    
    # Run optimization starting from the initial guess.
    x0 = [init_tx, init_ty, init_tz, init_rx, init_ry, init_rz]
    res = minimize(chamfer_distance, x0, method='Nelder-Mead', options={'maxiter': 100})
    optimal = res.x
    print("Optimization result:", optimal)
 
    slider_tx.set(optimal[0])
    slider_ty.set(optimal[1])
    slider_tz.set(optimal[2])
    slider_rx.set(np.rad2deg(optimal[3]))
    slider_ry.set(np.rad2deg(optimal[4]))
    slider_rz.set(np.rad2deg(optimal[5]))
 
    apply_transformation()
    print("Moved strawberry model to partial pointcloud's centroid.")
 
def save_transformation():
    current_filename=os.path.basename(partial_files[partial_index-1])
    identifier = current_filename.replace("segmented_", "").replace(".ply", "")
    
    # os.makedirs("transformation_matrix_", exist_ok=True)
    # np.save(f"transformation_matrix_/transformation_matrix_{identifier}.npy", transformation)
    # os.makedirs("transformation_orientation3_matrix", exist_ok=True)
    # np.save(f"transformation_orientation3_matrix/transformation_matrix_{identifier}.npy", transformation)
    os.makedirs("transformation_detection_matrix", exist_ok=True)
    np.save(f"transformation_detection_matrix/transformation_matrix_{identifier}.npy", transformation)

    transformed_points = (transformation[:3, :3] @ orig_points.T).T + transformation[:3, 3]
    pcd_save = o3d.geometry.PointCloud()
    pcd_save.points = o3d.utility.Vector3dVector(transformed_points)
    pcd_save.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
    # os.makedirs("transformation_strawberry_model_", exist_ok=True)
    # o3d.io.write_point_cloud(f"transformation_strawberry_model_/transformation_strawberry_model_{identifier}.ply", pcd_save)
    # os.makedirs("transformation_orientation3_model", exist_ok=True)
    # o3d.io.write_point_cloud(f"transformation_orientation3_model/transformation_strawberry_model_{identifier}.ply", pcd_save)
    os.makedirs("transformation_detection_model", exist_ok=True)
    o3d.io.write_point_cloud(f"transformation_detection_model/transformation_strawberry_model_{identifier}.ply", pcd_save)

    print("Saved transformation matrix and model for", identifier)

    
def load_transformation():
    current_filename = os.path.basename(partial_files[partial_index-1])
    identifier = current_filename.replace("segmented_", "").replace(".ply", "")
    path = f"transformation_detection_matrix/transformation_matrix_{identifier}.npy"
    if os.path.exists(path):
        loaded_T = np.load(path)
        # Update sliders with translation
        slider_tx.set(loaded_T[0, 3])
        slider_ty.set(loaded_T[1, 3])
        slider_tz.set(loaded_T[2, 3])
            # Compute Euler angles (assume rotation order x-y-z)
        R = loaded_T[:3, :3]
        sy = np.sqrt(R[0, 0]**2 + R[1, 0]**2)
        if sy > 1e-6:
            rx = np.arctan2(R[2, 1], R[2, 2])
            ry = np.arctan2(-R[2, 0], sy)
            rz = np.arctan2(R[1, 0], R[0, 0])
        else:
            rx = np.arctan2(-R[1, 2], R[1, 1])
            ry = np.arctan2(-R[2, 0], sy)
            rz = 0
        slider_rx.set(np.rad2deg(rx))
        slider_ry.set(np.rad2deg(ry))
        slider_rz.set(np.rad2deg(rz))
            # Apply the loaded transformation
        global transformation
        transformation = loaded_T
        transformed_points = (loaded_T[:3, :3] @ orig_points.T).T + loaded_T[:3, 3]
        pcd_display.points = o3d.utility.Vector3dVector(transformed_points)
        pcd_display.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
        vis.update_geometry(pcd_display)
        apply_transformation()

        print("Loaded transformation from", path)
    else:
        print("No saved transformation file for", identifier)
    
 
# Process All function remains unchanged
def process_all():

    pattern = os.path.join(folder, "segmented_*.ply")
    files = sorted(glob.glob(pattern))
    for filepath in files:
        print("Processing", filepath)
        current_partial_pcd = o3d.io.read_point_cloud(filepath)
        current_points = np.asarray(current_partial_pcd.points)
        current_partial_pcd.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (current_points.shape[0], 1)))
        global partial_points, partial_pcd
        partial_points = current_points
        try:
            vis.remove_geometry(partial_pcd)
        except:
            pass
        partial_pcd = current_partial_pcd
        vis.add_geometry(partial_pcd)
        strawberry_centroid = np.mean(orig_points, axis=0)
        partial_centroid = np.mean(partial_points, axis=0)
        translation = partial_centroid - strawberry_centroid
        slider_tx.set(translation[0])
        slider_ty.set(translation[1])
        slider_tz.set(translation[2])
        slider_rx.set(90)
        slider_ry.set(0)
        slider_rz.set(0)
        apply_transformation()
       
        identifier = os.path.splitext(os.path.basename(filepath))[0]
        # identifier = os.path.basename(filename)
 
        save_transformation(identifier)
        vis.poll_events()
 
# Create a sorted list of partial pointcloud files and an index for going to the next partial.
# partial_files = sorted(glob.glob(os.path.join(folder, "segmented_*.ply")))
partial_files = sorted(glob.glob(os.path.join(folder, "filtered_points_*.ply")))
partial_index = 1   # Since the first file is already loaded
 
def next_partial():
    global partial_files, partial_index, partial_pcd, partial_points, vis
    if partial_index >= len(partial_files):
        print("No more partial pointclouds.")
        return
    filename = partial_files[partial_index]
    partial_index += 1
    print("Loading", filename)
    # Read the new partial
    new_partial = o3d.io.read_point_cloud(filename)
    new_partial_points = np.asarray(new_partial.points)
    new_partial.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (new_partial_points.shape[0], 1)))
    partial_points = new_partial_points
    try:
        vis.remove_geometry(partial_pcd)
    except:
        pass
    partial_pcd = new_partial
    # Update sliders so that the strawberry model is centered on the partial
    strawberry_centroid = np.mean(orig_points, axis=0)
    partial_centroid = np.mean(partial_points, axis=0)
    translation = partial_centroid - strawberry_centroid
    slider_tx.set(translation[0])
    slider_ty.set(translation[1])
    slider_tz.set(translation[2])
    slider_rx.set(90)
    slider_ry.set(0)
    slider_rz.set(0)
    apply_transformation()
    # Re-create the visualizer window with the new title (the title now shows the partial file name)
    new_title = f"Visualizer - {os.path.basename(filename)}"
 
    vis.destroy_window()
    vis.create_window(new_title, width=1920, height=2160, left=2000)
    vis.add_geometry(pcd_display)
    vis.add_geometry(partial_pcd)
    # vis.add_geometry(coordinate_frame)
    print("Updated visualizer with new title.")
 
def update_visualizer():
    vis.poll_events()
    vis.update_renderer()
    root.after(50, update_visualizer)
 
def create_slider_frame(parent, label_text, slider_from, slider_to, resolution):
    frame = tk.Frame(parent)
    frame.pack(pady=2)
    tk.Label(frame, text=label_text).pack(side=tk.LEFT)
    slider = tk.Scale(frame, from_=slider_from, to=slider_to, resolution=resolution,
                      orient=tk.HORIZONTAL, length=200, command=lambda x: apply_transformation())
    slider.pack(side=tk.LEFT)
    btn_minus = tk.Button(frame, text="-", command=lambda: slider.set(slider.get() - resolution))
    btn_minus.pack(side=tk.LEFT, padx=2)
    btn_plus = tk.Button(frame, text="+", command=lambda: slider.set(slider.get() + resolution))
    btn_plus.pack(side=tk.LEFT, padx=2)
    return slider
 
root = tk.Tk()
root.title("Strawberry Transformation")
 
slider_tx = create_slider_frame(root, "Translation X", -1, 1, 0.001)
slider_ty = create_slider_frame(root, "Translation Y", -1, 1, 0.001)
slider_tz = create_slider_frame(root, "Translation Z", -1, 1, 0.001)
slider_rx = create_slider_frame(root, "Rotation X (deg)", -180, 180, 0.5)
slider_ry = create_slider_frame(root, "Rotation Y (deg)", -180, 180, 0.5)
slider_rz = create_slider_frame(root, "Rotation Z (deg)", -180, 180, 0.5)
 
btn_move = tk.Button(root, text="Move to Centroid", command=move_to_centroid)
btn_move.pack(pady=5)
 
btn_load = tk.Button(root, text="Load Transformation", command=load_transformation)
btn_load.pack(pady=5) 
 
 
btn_save = tk.Button(root, text="Save Transformation", command=lambda: save_transformation())
btn_save.pack(pady=5)
btn_process_all = tk.Button(root, text="Process All Partials", command=process_all)
btn_process_all.pack(pady=5)
 
# New button to go to the next partial pointcloud
btn_next = tk.Button(root, text="Next Partial", command=next_partial)
btn_next.pack(pady=5)
 
root.after(50, update_visualizer)
root.mainloop()

convert malak labels

In [ ]:
import os
import numpy as np
import open3d as o3d

# Define folders
seg_folder = "data/detection_data_539"
model_folder = "data/transformation_detection_539_model"
matrix_folder = "data/transformation_detection_539_matrix"
data_folder = "data"

# Use this prefix to extract the identifier.
prefix = "transformation_strawberry_model_filtered_points_"

# Start index from 1000
idx = 1133

# Number of augmentations per original file (set to 1 here)
num_augmentations = 2

# Loop through all model files in model_folder
for model_filename in os.listdir(model_folder):
    if model_filename.endswith(".ply") and model_filename.startswith(prefix):
        # Expected model file format:
        # "transformation_strawberry_model_filtered_points_<identifier>.ply"
        identifier = model_filename[len(prefix):].replace(".ply", "")
        
        # Construct file paths using the identifier
        src_file = os.path.join(seg_folder, f"filtered_points_{identifier}.ply")
        matrix_file = os.path.join(matrix_folder, f"transformation_matrix_filtered_points_{identifier}.npy")
        model_file = os.path.join(model_folder, model_filename)
        
        # Read the source point cloud, the transformed model, and load the ground truth transformation matrix
        src_pcd = o3d.io.read_point_cloud(src_file)
        model_transformed_pcd = o3d.io.read_point_cloud(model_file)
        transform_gt = np.load(matrix_file)
        

        # Get 512 points for original data
        if len(src_pcd.points) >= 512:
            downsampled_pcd = src_pcd.farthest_point_down_sample(512)
            src_points = np.asarray(downsampled_pcd.points)
        else:
            original_points = np.asarray(src_pcd.points)
            repeats_needed = int(np.ceil(512 / len(original_points)))
            src_points = np.tile(original_points, (repeats_needed, 1))[:512]

        # # Convert point clouds to numpy arrays
        # src_points = np.asarray(src_pcd.points)
        model_transformed_points = np.asarray(model_transformed_pcd.points)
        
        # Save the original data in the specified format
        output_path = os.path.join(data_folder, f"processed_dataset_{idx}.npz")
        np.savez(output_path,
                 src_pcd=src_points,
                 transform_gt=transform_gt,
                 model_pcd_transformed=model_transformed_points)
        print(f"Saved evaluation data to {output_path}")
        idx += 1
        
        # Create and save augmented version(s)
        for aug in range(num_augmentations):
            translation = np.random.uniform(-1, 1, 3)
            src_aug = src_points + translation
            model_transformed_aug = model_transformed_points + translation
            
            aug_output_path = os.path.join(data_folder, f"processed_dataset_{idx}.npz")
            np.savez(aug_output_path,
                     src_pcd=src_aug,
                     transform_gt=transform_gt,
                     model_pcd_transformed=model_transformed_aug)
            print(f"Saved augmented evaluation data to {aug_output_path}")
            idx += 1
  

vis lablels

In [ ]:
import glob, os
import open3d as o3d
folder = "data/segmented"
out_folder = "transformation_detection_model"
frame = "0100"
detection_num = "1"
in_file = os.path.join(folder, f"filtered_points_{frame}_detection_{detection_num}.ply")
out_file = os.path.join(out_folder, f"transformation_strawberry_model_filtered_points_{frame}_detection_{detection_num}.ply")

pcd_input = o3d.io.read_point_cloud(in_file)
pcd_model = o3d.io.read_point_cloud(out_file)


pcd_input.paint_uniform_color([0, 1, 0])   # Green for the input point cloud
pcd_model.paint_uniform_color([0, 0, 1])   # Blue for the transformed model

o3d.visualization.draw_geometries([pcd_input, pcd_model])


# partial_files = sorted(glob.glob(os.path.join(folder, "filtered_points_*.ply")))
# partial_index = 1   # Since the first file is already loaded

convert labels to stuff

In [ ]:
import os
import numpy as np
import open3d as o3d

# Define folders
seg_folder = "data/segmented"
model_folder = "transformation_detection_model"
matrix_folder = "transformation_detection_matrix"
data_folder = "data"

# Use this prefix to extract the identifier.
prefix = "transformation_strawberry_model_filtered_points_"

# Start index from 1000
idx = 1000

# Number of augmentations per original file (set to 1 here)
num_augmentations = 2

# Loop through all model files in model_folder
for model_filename in os.listdir(model_folder):
    if model_filename.endswith(".ply") and model_filename.startswith(prefix):
        # Expected model file format:
        # "transformation_strawberry_model_filtered_points_<identifier>.ply"
        identifier = model_filename[len(prefix):].replace(".ply", "")
        
        # Construct file paths using the identifier
        src_file = os.path.join(seg_folder, f"filtered_points_{identifier}.ply")
        matrix_file = os.path.join(matrix_folder, f"transformation_matrix_filtered_points_{identifier}.npy")
        model_file = os.path.join(model_folder, model_filename)
        
        # Read the source point cloud, the transformed model, and load the ground truth transformation matrix
        src_pcd = o3d.io.read_point_cloud(src_file)
         # Process original points
        

        # Get 512 points for original data
        if len(src_pcd.points) >= 512:
            downsampled_pcd = src_pcd.farthest_point_down_sample(512)
            src_points = np.asarray(downsampled_pcd.points)
        else:
            original_points = np.asarray(src_pcd.points)
            repeats_needed = int(np.ceil(512 / len(original_points)))
            src_points = np.tile(original_points, (repeats_needed, 1))[:512]

        model_transformed_pcd = o3d.io.read_point_cloud(model_file)
        transform_gt = np.load(matrix_file)
        
        # Convert point clouds to numpy arrays
        # src_points = np.asarray(src_pcd.points)

        model_transformed_points = np.asarray(model_transformed_pcd.points)
        
        # Save the original data in the specified format
        output_path = os.path.join(data_folder, f"processed_dataset_{idx}.npz")
        np.savez(output_path,
                 src_pcd=src_points,
                 transform_gt=transform_gt,
                 model_pcd_transformed=model_transformed_points)
        print(f"Saved evaluation data to {output_path}")
        idx += 1
        
        # Create and save augmented version(s)
        for aug in range(num_augmentations):
            translation = np.random.uniform(-1, 1, 3)
            src_aug = src_points + translation
            model_transformed_aug = model_transformed_points + translation
            
            aug_output_path = os.path.join(data_folder, f"processed_dataset_{idx}.npz")
            np.savez(aug_output_path,
                     src_pcd=src_aug,
                     transform_gt=transform_gt,
                     model_pcd_transformed=model_transformed_aug)
            print(f"Saved augmented evaluation data to {aug_output_path}")
            idx += 1
  

visualize converted stuff

In [16]:
eval_file = os.path.join(data_folder, "processed_dataset_1032.npz")
data_eval = np.load(eval_file)

src_points = data_eval['src_pcd']
model_points = data_eval['model_pcd_transformed']

pcd_src = o3d.geometry.PointCloud()
pcd_src.points = o3d.utility.Vector3dVector(src_points)
pcd_src.paint_uniform_color([0, 1, 0])  # green for source

pcd_model = o3d.geometry.PointCloud()
pcd_model.points = o3d.utility.Vector3dVector(model_points)
pcd_model.paint_uniform_color([0, 0, 1])  # blue for transformed model

o3d.visualization.draw_geometries([pcd_src, pcd_model])